Search prices on Google Shopping e Buscape

In [9]:
#start setup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
import time
import win32com.client as win32

nav = webdriver.Chrome()

#import search terms
search_terms = pd.read_excel("buscas.xlsx")
display(search_terms)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64 gb,mini watch,3000,3500
1,rtx 3060,zota galax,4000,4500


Define a função de busca Google Shopping e Buscapé

In [6]:
def busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo):

    #entrar no google
    nav.get("https://www.google.com")

    #tratar os valores da tabela
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_produto = produto.split(" ")
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)
    
    #Pesquisar o nome do produto
    nav.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(produto)
    nav.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

    #clicar na barra Shopping
    elementos = nav.find_elements(By.CLASS_NAME, 'hdtb-mitem')
    for item in elementos:
        if "Shopping" in item.text:
            item.click()
            break
            
    #pegar a lista de resultados da busca no google shopping
    lista_resultados = nav.find_elements(By.CLASS_NAME, 'sh-dgr__content')
    
    #para cada resultado, verifica se corresponde a todas as condições
    
    lista_ofertas = [] #lista de resposta

    for result in lista_resultados:
        nome = result.find_element(By.CLASS_NAME, 'Xjkr3b').text
        nome = nome.lower()

        #verifica se há algum termo banido
        tem_termos_banidos = False
        for palavra in lista_termos_banidos:
            if palavra in nome:
                tem_termos_banidos = True
                
        #verifica se o nome está de acordo
        tem_todos_termos_produto = True
        for palavra in lista_termos_produto:
            if palavra not in nome:
                tem_todos_termos_produto = False
                
        
        if not tem_termos_banidos and tem_todos_termos_produto: #verifica os nomes
            try:
                preco = result.find_element(By.CLASS_NAME, 'a8Pemb').text
                preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
                preco = float(preco)
            except:
                    continue

            if preco_minimo <= preco <= preco_maximo: #verifica se o preço está de acordo           
                elemento_link = result.find_element(By.CLASS_NAME, 'aULzUe')
                elemento_pai = elemento_link.find_element(By.XPATH, '..')
                link = elemento_pai.get_attribute('href')
                lista_ofertas.append((nome, preco, link))
    return lista_ofertas
        
def busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo):
    
    #entrar no buscape
    nav.get("https://www.buscape.com.br/")
    
    #tratar valores
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_produto = produto.split(" ")
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)
    
    #insere a busca
    nav.find_element(By.CLASS_NAME, 'search-bar__text-box').send_keys(produto, Keys.ENTER)
    
    time.sleep(5)  
    lista_resultados = nav.find_elements(By.CLASS_NAME, 'Cell_Content__1630r')
    
    lista_ofertas = []    
    for resultado in lista_resultados:
        try:
            preco = resultado.find_element(By.CLASS_NAME, 'CellPrice_MainValue__3s0iP').text
            nome = resultado.get_attribute('title')
            nome = nome.lower()
            link = resultado.get_attribute('href')
            
            #verifica se há algum termo banido
            tem_termos_banidos = False
            for palavra in lista_termos_banidos:
                if palavra in nome:
                    tem_termos_banidos = True
                
            #verifica se o nome está de acordo
            tem_todos_termos_produto = True
            for palavra in lista_termos_produto:
                if palavra not in nome:
                    tem_todos_termos_produto = False
                    
            if not tem_termos_banidos and tem_todos_termos_produto: #verifica os nomes
                try:
                    preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
                    preco = float(preco)
                except:
                    continue

                if preco_minimo <= preco <= preco_maximo: #verifica se o preço está de acordo           
                    lista_ofertas.append((nome, preco, link))
                        
        except:
            pass
        
    return lista_ofertas


def send_mail(towho, attach):
    
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = towho
    mail.Subject = 'Produto(s) Encontrados(s) na faixa de preço desejada'
    mail.HTMLBody = f"""
    <p>To whom it may concern,</p>
    <p>We have finde somes itens in the price range. For details, follow the table below:</p>
    {attach.to_html(index=False)}
    <p>Please let me know if you have any questions.</p>
    <p>Best regards.</p>
    <p>Mr. Robot</p>
    """
    
    mail.Send()

Contrução das variáveis para ofertas

In [3]:
tabela_ofertas = pd.DataFrame()

for linha in search_terms.index:
    produto = search_terms.loc[linha, "Nome"]
    termos_banidos = search_terms.loc[linha, "Termos banidos"]
    preco_minimo = search_terms.loc[linha, "Preço mínimo"]
    preco_maximo = search_terms.loc[linha, "Preço máximo"]
    
    lista_ofertas_google_shopping = busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_google_shopping:
        tabela_google_shopping = pd.DataFrame(lista_ofertas_google_shopping, columns=['produto', 'preço', 'link'])
        tabela_ofertas = tabela_ofertas.append(tabela_google_shopping)
    else:
        tabela_google_shopping = None

    lista_ofertas_buscape = busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_buscape:
        tabela_buscape = pd.DataFrame(lista_ofertas_buscape, columns=['produto', 'preço', 'link'])
        tabela_ofertas = tabela_ofertas.append(tabela_buscape)
    else:
        tabela_buscape = None
    

Exportas a base de dados para o Excel

In [4]:
tabela_ofertas = tabela_ofertas.reset_index(drop=True)
tabela_ofertas.to_excel("Ofertas.xlsx", index=False)

Enviando o e-mail

In [8]:
if len(tabela_ofertas.index) > 0:
    send_mail('fernandodtschmitz@gmail.com', tabela_ofertas)
    
nav.quit()    